In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import copy
import os
from tqdm import tqdm

In [2]:
loadPath = "./Data/"
loadFile = "SteamAPI_NMS_reviews.xlsx" ## Base file name

In [3]:
dfList = []
loadFileCounter = 0

fileFound = True
while fileFound == True:
    loadFileCounter+=1
    
    ## Proper filename for current loadfile number
    name, fileExt = loadFile.split('.')
    currentloadFile = name + str(loadFileCounter).zfill(3) + '.' + fileExt
    try:
        tmpdf = pd.read_excel(loadPath+currentloadFile,index_col=0)
        fileFound = True
        dfList.append(tmpdf)
    except:
        print(f"No further files detected. Final file {name + str(loadFileCounter-1).zfill(3) + '.' + fileExt}")
        fileFound = False

No further files detected. Final file SteamAPI_NMS_reviews005.xlsx


In [6]:
df = pd.concat(dfList,ignore_index=False)
df

,recommendationid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
0,198664720,0,1,250,172,243.0,1751392649,english,I liked every thing,2025-07-01 17:47:05,2025-07-01 17:47:05,True,0,0,0.500000,0,True,False,False,False
1,198664615,0,16,16380,1338,16380.0,1751392203,swedish,Good game,2025-07-01 17:45:39,2025-07-01 17:45:39,True,0,0,0.500000,0,True,False,False,False
2,198663932,0,33,14,0,14.0,1672123150,english,No Man in Sky :(,2025-07-01 17:36:47,2025-07-01 17:36:47,True,0,0,0.500000,0,False,False,False,False
3,198663626,0,1,4320,2079,4320.0,1751197679,schinese,\u8fd9\u53ef\u592a\u597d\u73a9\u4e86\uff0c\u67...,2025-07-01 17:33:09,2025-07-01 17:33:09,True,0,0,0.500000,0,False,False,False,False
4,198663578,32,3,1275,1275,1275.0,1751391044,russian,\u043b\u0435\u0442\u0430\u0435\u0448\u044c \u0...,2025-07-01 17:32:26,2025-07-01 17:32:26,True,0,0,0.500000,0,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55180,24845536,56,3,1081,0,136.0,1742339118,schinese,\u671f\u5f85 \u505a\u6c99\u53d1,2016-08-12 17:16:27,2016-08-12 17:16:27,True,0,0,0.500000,0,False,False,False,False
55181,24845492,146,6,27187,0,293.0,1734065603,english,Pretty Darn Good,2016-08-12 17:14:28,2016-08-12 17:14:28,True,0,0,0.450378,0,True,False,False,False
55182,24845487,235,11,933,0,224.0,1618267916,english,Big frame drops and FOV feels too small. Other...,2016-08-12 17:14:03,2016-08-12 17:14:03,True,0,0,0.396006,0,True,False,False,False
55183,24845456,254,1,569,0,364.0,1723120964,tchinese,cannot play!!!!!!!!!!!!!!!!!!!,2016-08-12 17:12:26,2016-08-12 17:12:26,True,4,3,0.493138,0,True,False,False,False


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 355379 entries, 0 to 55184
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   recommendationid             355379 non-null  int64         
 1   num_games_owned              355379 non-null  int64         
 2   num_reviews                  355379 non-null  int64         
 3   playtime_forever             355379 non-null  int64         
 4   playtime_last_two_weeks      355379 non-null  int64         
 5   playtime_at_review           355246 non-null  float64       
 6   last_played                  355379 non-null  int64         
 7   language                     355379 non-null  object        
 8   review                       354299 non-null  object        
 9   timestamp_created            355379 non-null  datetime64[ns]
 10  timestamp_updated            355379 non-null  datetime64[ns]
 11  voted_up                     355

In [9]:
df.describe()

,recommendationid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,timestamp_created,timestamp_updated,votes_up,votes_funny,weighted_vote_score,comment_count
count,3.553790e+05,355379.000000,355379.000000,3.553790e+05,355379.000000,3.552460e+05,3.553790e+05,355379,355379,355379.000000,3.553790e+05,355379.000000,355379.000000
mean,9.489044e+07,143.274839,17.806668,1.079377e+04,41.935874,5.088608e+03,1.695329e+09,2020-12-29 15:12:40.979635712,2021-05-06 11:59:19.874978560,4.150628,7.251458e+04,0.504683,0.184904
min,2.484539e+07,0.000000,1.000000,0.000000e+00,0.000000,1.000000e+00,0.000000e+00,2016-08-12 17:08:41,2016-08-12 17:08:41,0.000000,0.000000e+00,0.058376,0.000000
25%,4.382008e+07,0.000000,3.000000,1.934000e+03,0.000000,6.730000e+02,1.668295e+09,2018-07-29 17:48:44.500000,2019-02-10 11:27:39,0.000000,0.000000e+00,0.500000,0.000000
50%,8.940440e+07,0.000000,8.000000,5.112000e+03,0.000000,1.939000e+03,1.723771e+09,2021-03-31 01:49:18,2021-08-31 06:13:32,0.000000,0.000000e+00,0.500000,0.000000
75%,1.455082e+08,158.000000,18.000000,1.143100e+04,0.000000,5.079000e+03,1.740872e+09,2023-09-02 08:54:23.500000,2023-11-22 12:58:25,1.000000,0.000000e+00,0.502488,0.000000
max,1.986647e+08,33469.000000,17160.000000,2.309235e+06,20085.000000,1.134557e+06,1.751397e+09,2025-07-01 17:47:05,2025-07-01 17:47:05,16997.000000,4.294967e+09,0.992365,1424.000000
std,5.568367e+07,385.025041,63.577084,2.420035e+04,372.508716,1.293489e+04,6.774500e+07,NaN,NaN,77.498810,1.764764e+07,0.043107,3.788062
